<a href="https://colab.research.google.com/github/formallinguist/formallinguist/blob/main/Prognoz_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
Data = pd.read_csv("/content/Reviews.csv")

In [ ]:
print (Data.head())

   Id  ...                                               Text
0   1  ...  I have bought several of the Vitality canned d...
1   2  ...  Product arrived labeled as Jumbo Salted Peanut...
2   3  ...  This is a confection that has been around a fe...
3   4  ...  If you are looking for the secret ingredient i...
4   5  ...  Great taffy at a great price.  There was a wid...

[5 rows x 10 columns]


In [ ]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122155 entries, 0 to 122154
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      122155 non-null  int64 
 1   ProductId               122155 non-null  object
 2   UserId                  122155 non-null  object
 3   ProfileName             122150 non-null  object
 4   HelpfulnessNumerator    122155 non-null  int64 
 5   HelpfulnessDenominator  122155 non-null  int64 
 6   Score                   122155 non-null  int64 
 7   Time                    122155 non-null  int64 
 8   Summary                 122150 non-null  object
 9   Text                    122155 non-null  object
dtypes: int64(5), object(5)
memory usage: 9.3+ MB


In [ ]:
Data.Score.value_counts()

5    76866
4    17690
1    11420
3     9552
2     6627
Name: Score, dtype: int64

In [ ]:
Data.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [ ]:
Data['Prediction']=Data.Score.apply(lambda x: "1" if x>3 else "0")

In [ ]:
Data.drop(columns = ["Id","ProductId","UserId","ProfileName","HelpfulnessNumerator","HelpfulnessDenominator","Time"], inplace = True)

In [ ]:
Data.head(4)

,Score,Summary,Text,Prediction
0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,4,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,2,Cough Medicine,If you are looking for the secret ingredient i...,0


In [ ]:
nltk.download ('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
Data ['Clean_Title'] = Data ['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in(stop_words)]))

In [ ]:
Data[['Text','Clean_Title']].head(9)

,Text,Clean_Title
0,I have bought several of the Vitality canned d...,I bought several Vitality canned dog food prod...
1,Product arrived labeled as Jumbo Salted Peanut...,Product arrived labeled Jumbo Salted Peanuts.....
2,This is a confection that has been around a fe...,"This confection around centuries. It light, pi..."
3,If you are looking for the secret ingredient i...,If looking secret ingredient Robitussin I beli...
4,Great taffy at a great price. There was a wid...,Great taffy great price. There wide assortment...
5,I got a wild hair for taffy and ordered this f...,I got wild hair taffy ordered five pound bag. ...
6,This saltwater taffy had great flavors and was...,This saltwater taffy great flavors soft chewy....
7,This taffy is so good. It is very soft and ch...,This taffy good. It soft chewy. The flavors am...
8,Right now I'm mostly just sprouting this so my...,Right I'm mostly sprouting cats eat grass. The...


In [ ]:
import re

In [ ]:
import string

TEXT CLEANING

In [ ]:
def text_clean_1(Clean_Title):
  Clean_Title = Clean_Title.lower()              
  Clean_Title = re.sub('\[.*?\]','',Clean_Title)
  Clean_Title = re.sub('[%s]' % re.escape(string.punctuation),'',Clean_Title)
  Clean_Title = re.sub('\w*\d\w*','',Clean_Title)
  return Clean_Title

  

In [ ]:
cleaned1 = lambda x: text_clean_1(x)

In [ ]:
Data['Clean_Title']= Data['Clean_Title'].apply(str)

In [ ]:
Data['cleaned_Text'] = pd.DataFrame(Data.Clean_Title.apply(cleaned1))
Data.head(4)

,Score,Summary,Text,Prediction,Clean_Title,cleaned_Text
0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,1,I bought several Vitality canned dog food prod...,i bought several vitality canned dog food prod...
1,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0,Product arrived labeled Jumbo Salted Peanuts.....,product arrived labeled jumbo salted peanutsth...
2,4,"""Delight"" says it all",This is a confection that has been around a fe...,1,"This confection around centuries. It light, pi...",this confection around centuries it light pill...
3,2,Cough Medicine,If you are looking for the secret ingredient i...,0,If looking secret ingredient Robitussin I beli...,if looking secret ingredient robitussin i beli...


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
independent_var = Data.cleaned_Text

In [ ]:
dependent_var = Data.Prediction

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(independent_var,dependent_var,test_size=0.2)

In [ ]:
print('x_train:',len(x_train))
print('x_test:',len(x_test))
print('y_train:',len(y_train))
print('y_test:',len(y_test))


x_train: 97724
x_test: 24431
y_train: 97724
y_test: 24431


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
tvec = TfidfVectorizer()
clf2 = LogisticRegression(solver='lbfgs')

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
model = Pipeline([('vectorizer',tvec),('classifier',clf2)])

In [ ]:
model.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', LogisticRegression())])

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
predictions = model.predict(x_test)

In [ ]:
 confusion_matrix(predictions, y_test)

array([[ 3458,   731],
       [ 2139, 18103]])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
print ("Accuracy:", accuracy_score(predictions,y_test))

Accuracy: 0.8825262985551144


In [ ]:
print ("Precision:", precision_score(predictions,y_test, average = 'weighted'))

Precision: 0.9023145011293355


In [ ]:
print ("Recall:", recall_score(predictions,y_test, average = 'weighted'))

Recall: 0.8825262985551144


MODEL

In [ ]:
Review =['it is a good product']
result = model.predict(Review)
x = (int(result))
if x > 0:
  print(x,"is a positive review")
else:
  print(x,"is a negative review")